In [3]:
import numpy as np
import os
import keras
import tempfile
import tensorflow as tf
from tensorflow.keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import preprocess_input
from tensorflow_model_optimization.sparsity import keras as sparsity

In [4]:
model = tf.keras.models.load_model('model_MobileNet.h5')

In [5]:
nsfw_images = [fn for fn in os.listdir(r'D:\Data\Train\NSFW') if fn.endswith('.jpg')]
sfw_images = [fn for fn in os.listdir(r'D:\Data\Train\SFW') if fn.endswith('.jpg')]
train_dir = r'D:\Data\Train'
validation_dir = r'D:\Data\Validation'

def prepare_image(file):
    img_path = ''
    img = image.load_img(img_path + file, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory(train_dir,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=30,
                                                 class_mode='categorical',
                                                 shuffle=True)

test_datagen = ImageDataGenerator( rescale = 1.0/255.)

validation_generator =test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 30,
                                                          class_mode  = 'categorical', 
                                                          target_size = (224, 224))

Found 6394 images belonging to 2 classes.
Found 2137 images belonging to 2 classes.


In [6]:
epochs = 10
end_step = np.ceil(1.0 * 6394 / 30).astype(np.int32) * epochs
print(end_step)

2140


In [8]:
new_pruning_parameters = {
    'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.50,
                                                final_sparsity=0.90,
                                                begin_step=0,
                                                end_step=end_step,
                                                frequency=100)
}

new_pruned_model = sparsity.prune_low_magnitude(model, **new_pruning_parameters)
new_pruned_model.summary()

Instructions for updating:
Please use `layer.add_weight` method instead.
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
prune_low_magnitude_conv1_pa (None, 225, 225, 3)       1         
_________________________________________________________________
prune_low_magnitude_conv1 (P (None, 112, 112, 32)      1730      
_________________________________________________________________
prune_low_magnitude_conv1_bn (None, 112, 112, 32)      129       
_________________________________________________________________
prune_low_magnitude_conv1_re (None, 112, 112, 32)      1         
_________________________________________________________________
prune_low_magnitude_conv_dw_ (None, 112, 112, 32)      289       
_______________________________________________

In [15]:
new_pruned_model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy']
)

In [16]:
logdir = tempfile.mkdtemp()

callbacks = [
    sparsity.UpdatePruningStep(),
    sparsity.PruningSummaries(log_dir=logdir, profile_batch=0)
]

new_pruned_model.fit(train_generator, 
                    validation_data=validation_generator,
                    steps_per_epoch=int(6394/30),
                    validation_steps=int(2137/30),
                    epochs = epochs,
                    callbacks = callbacks)

Epoch 1/10
  3/213 [..............................] - ETA: 8:08 - loss: 0.1576 - accuracy: 0.9556

C:\Users\Harshinee Sriram\anaconda3\lib\site-packages\PIL\Image.py:2832: DecompressionBombWarning: Image size (147659204 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


213/213 [==============================] - 813s 4s/step - loss: 0.1905 - accuracy: 0.9365 - val_loss: 0.6549 - val_accuracy: 0.7906
Epoch 2/10
213/213 [==============================] - 398s 2s/step - loss: 0.1139 - accuracy: 0.9613 - val_loss: 0.3153 - val_accuracy: 0.8915
Epoch 3/10
213/213 [==============================] - 391s 2s/step - loss: 0.1099 - accuracy: 0.9624 - val_loss: 1.6244 - val_accuracy: 0.5934
Epoch 4/10
213/213 [==============================] - 378s 2s/step - loss: 0.0945 - accuracy: 0.9665 - val_loss: 1.4174 - val_accuracy: 0.6376
Epoch 5/10
213/213 [==============================] - 399s 2s/step - loss: 0.1210 - accuracy: 0.9554 - val_loss: 0.7994 - val_accuracy: 0.6512
Epoch 6/10
213/213 [==============================] - 391s 2s/step - loss: 0.1059 - accuracy: 0.9620 - val_loss: 0.7637 - val_accuracy: 0.7028
Epoch 7/10
213/213 [==============================] - 383s 2s/step - loss: 0.0876 - accuracy: 0.9661 - val_loss: 2.3271 - val_accuracy: 0.5371
Epoch 8/10

In [17]:
final_model = sparsity.strip_pruning(new_pruned_model)
final_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)     

In [18]:
final_model.save('compressed.h5')
model = load_model('compressed.h5')
for i, w in enumerate(model.get_weights()):
    print(
        "{} -- Total:{}, Zeros: {:.2f}%".format(
            model.weights[i].name, w.size, np.sum(w == 0) / w.size * 100
        )
    )

conv1/kernel:0 -- Total:864, Zeros: 90.05%
conv1_bn/gamma:0 -- Total:32, Zeros: 0.00%
conv1_bn/beta:0 -- Total:32, Zeros: 0.00%
conv1_bn/moving_mean:0 -- Total:32, Zeros: 0.00%
conv1_bn/moving_variance:0 -- Total:32, Zeros: 0.00%
conv_dw_1/depthwise_kernel:0 -- Total:288, Zeros: 0.00%
conv_dw_1_bn/gamma:0 -- Total:32, Zeros: 0.00%
conv_dw_1_bn/beta:0 -- Total:32, Zeros: 0.00%
conv_dw_1_bn/moving_mean:0 -- Total:32, Zeros: 0.00%
conv_dw_1_bn/moving_variance:0 -- Total:32, Zeros: 0.00%
conv_pw_1/kernel:0 -- Total:2048, Zeros: 89.99%
conv_pw_1_bn/gamma:0 -- Total:64, Zeros: 0.00%
conv_pw_1_bn/beta:0 -- Total:64, Zeros: 0.00%
conv_pw_1_bn/moving_mean:0 -- Total:64, Zeros: 0.00%
conv_pw_1_bn/moving_variance:0 -- Total:64, Zeros: 0.00%
conv_dw_2/depthwise_kernel:0 -- Total:576, Zeros: 0.00%
conv_dw_2_bn/gamma:0 -- Total:64, Zeros: 0.00%
conv_dw_2_bn/beta:0 -- Total:64, Zeros: 0.00%
conv_dw_2_bn/moving_mean:0 -- Total:64, Zeros: 0.00%
conv_dw_2_bn/moving_variance:0 -- Total:64, Zeros: 0.00%
c